# DICE — Notebook 1 (Basics)

This notebook is a **minimal teaching scaffold** to help you (and students) simulate the DICE-style model step by step and produce a few baseline plots.  
Scope (kept intentionally small):
- Load the provided `DICE.py` (parameters, state-update, light planner scaffolding).
- Build the time grid and initial state matrix.
- Pick **one simple control path** (abatement `μ_t`) and simulate.
- Plot **one endogenous path** (e.g., temperature `T_AT`) and **one exogenous path** (e.g., non-CO₂ forcing `F_EX`).

> Advanced/optimization content is **not** the focus here; see Notebook 2 for the control problem.


## 0) Setup

In [7]:
# Standard imports
import sys, math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


## 1) Import the DICE module & inspect parameters

In [1]:
from DICE import *

# Create a default parameter object
p = Params()
print(p)
print("\nNumber of periods nT =", p.nT, "with step Δ =", p.Delta, "years")
print("Time runs from", p.t0, "to", p.tT)


Params(Delta=5, t0=2015, tT=2100, nT=18, rho=0.015, gamma=1.45, deltaK=0.1, alpha=0.3, A0=5.115, gA=0.076, deltaA=0.005, k0=220.0, lg=0.134, Linf=10500, L0=7403, gsig=0.0152, deltasig=0.0, sigma0=0.3503, pb=550, theta2=2.6, deltapb=0.025, a2=0.00236, a3=2, a4=0, a5=0, a6=0, F2XCO2=3.3613, T2XCO2=3.1, xi=0.2727272727272727, mat=588, mup=360, mlo=1720, b12=0.024, b23=0.0014, c1=0.0201, c3=0.0176, c4=0.005, T_AT0=0.85, T_LO0=0.007, M_AT0=851, M_UP0=460, M_LO0=1740, f1=1.0, f0=0.5, tf=17.0, EL0=2.6, deltaEL=0.115, s_lower=0.0, s_upper=1.0, tolx=1e-10, toly=0.001)

Number of periods nT = 18 with step Δ = 5 years
Time runs from 2015 to 2100


## 2) Build initial state matrix
### Time Structure and Variables in the DICE Model

Before setting up the numerical solution, let us clarify the time indexing and the role of variables:

- We work in **discrete time** with finite horizon $T$.
- Periods are indexed by $i=0,1,2,\dots,I$ with time step $\Delta$ years.
- Calendar time is $t(i) = t_0 + i\Delta$.

Thus, the time grid is:
$$
t = \{t_0, t_0 + \Delta, t_0 + 2\Delta, \dots, T\}.
$$

- Variables can be written as $Y_t$ (calendar time) or $Y(i)$ (discrete index).

### Setting exogenous states
Exogenous state variables are given by $x_t \in \mathbb{R}^{N_x}$: productivity $A_t$, carbon intensity $\sigma_t$, radiative forcing $F_t^{EX}$, land-use emissions $E_t^{land}$, population $L_t$, etc.

They follow an exogenous process:
  $$
  x_t = g(x_{t-1}), \quad g: \mathbb{R}^{N_x} \to \mathbb{R}^{N_x}.
  $$

They are next stored in to $y$ matrix, other variabels are yet left NaN.

In [5]:
# Initialize the simulation matrix with exogenous drivers and initial states
sim = init_states(p)

# Convert to a tidy DataFrame for inspection (optional)
df0 = mat_to_df(sim, p)
df0.head()

NameError: name 'init_states' is not defined

## 3) Choose a simple control path, then simulate

The vector of  **Endogenous states/flows** $y_t \in \mathbb{R}^{N_y}$: carbon stocks $(M_t^{AT}, M_t^{UP}, M_t^{LO})$, temperatures $(T_t^{AT}, T_t^{OC})$, output $Y_t$, capital $K_t$, etc. They evolve according to
  $$
  y_t = f(y_{t-1}, x_{t-1}, z_t), \quad f: \mathbb{R}^{N_y}\times \mathbb{R}^{N_x}\times \mathbb{R}^2 \to \mathbb{R}^{N_y}.
  $$

By setting **decision variables (controls)** $z_t = [\mu_t, s_t]' \in \mathbb{R}^2$: abatement rate and saving rate, we are able to numerically solve the path of the economy.


### Numerical Resolution (Open Loop)
Under Assumptions (1) and (2), decisions are exogenous and the model can be solved recursively:
1. Update exogenous states: $x_t = g(x_{t-1})$.
2. Compute endogenous states: $y_t = f(y_{t-1}, x_{t-1}, z_t)$.

This provides the full trajectory $(x_t, y_t, z_t)$ for $t = t_0, \dots, T$.

### In this example
For this **basics** notebook we avoid optimization. We simply **feed exogenous controls**, then call the **state updater**:
- Fix saving rate `s_t` to a constant (e.g., 20%).
- Set a **ramp** for the abatement rate `μ_t` to 3%  (purely illustrative).


In [ ]:
# Time indices to update (DICE arrays start at t=1 for transitions)
timevec = range(1, p.nT)

# 3.a) Controls: saving and abatement (illustrative — not optimal)
sim[1:, p.i_s]  = 0.20  # constant saving rate at 20%
# Start abatement at 3%
sim[1:, p.i_mu] =0.03

# 3.b) Propagate the system forward using the update function
sim = update_path(sim, timevec, p)

# Show a couple lines to confirm variables updated (e.g., Y, C, T_AT, F_EX)
cols_to_show = ['time','s','mu','Y','C','T_AT','F_EX','E_land']
pd.DataFrame(sim, columns=p.col)[cols_to_show].head(8)


## 4) Plot: one endogenous path and one exogenous driver

In [ ]:
years = sim[:, p.i_time]

# Endogenous example: atmospheric temperature T_AT
plt.figure()
plt.plot(years, sim[:, p.i_T_AT], linewidth=2)
plt.xlabel("Year"); plt.ylabel("Atmospheric temperature (°C)")
plt.title("Endogenous path: $T_{AT}$")
plt.grid(True)
plt.show()

# Exogenous example: non-CO₂ forcing F_EX
plt.figure()
plt.plot(years, sim[:, p.i_F_EX], linewidth=2)
plt.xlabel("Year"); plt.ylabel("Non-CO$_2$ forcing (W/m$^2$)")
plt.title("Exogenous driver: $F_{EX}$")
plt.grid(True)
plt.show()


## 5) Optional: quick look at Output and Consumption

In [ ]:
plt.figure()
plt.plot(years, sim[:, p.i_Q], linewidth=2, label="Output Q")
plt.plot(years, sim[:, p.i_C], linewidth=2, label="Consumption C")
plt.xlabel("Year"); plt.ylabel("Level (model units)")
plt.title("Output and Consumption (illustrative controls)")
plt.grid(True); plt.legend()
plt.show()


## 6) Question: Compare a laissez-faire scenario (no abatement $\mu=0$) against a green transition scenario (abatement starting at 0 in 2020 and reaching 100\% linearly in 2050). Report in a graph.

In [ ]:
# Your code here. Here is an help:
sim_BAU = sim.copy()
sim_BAU[1:, p.i_s]  = 0.20  # constant saving rate at 20%
sim_BAU[1:, p.i_mu] = 0